In [4]:
from ftplib import FTP
import os
import gzip
import xml.etree.ElementTree as ET
import pandas as pd

In [13]:
save_path = "./pubmed_data/"

data = []
for file in os.listdir(save_path):
    if file.endswith('.xml'):
        tree = ET.parse(os.path.join(save_path, file))
        root = tree.getroot()
        
        for article in root.findall('.//PubmedArticle'):
            language = article.find('.//Language').text if article.find('.//Language') is not None else None
            abstract = article.find('.//AbstractText').text if article.find('.//AbstractText') is not None else None
            
            if language and language.lower() == 'eng' and abstract:
                pub_id = article.find('.//PMID').text
                article_title = article.find('.//ArticleTitle').text if article.find('.//ArticleTitle') is not None else None
                year = article.find('.//DateCompleted')
                year = year.find('.//Year').text if year is not None else None
                journal = article.find('.//Journal')
                journal_title = journal.find('.//Title').text if journal is not None else None
                mesh_descriptor = [mesh_heading.text for mesh_heading in article.findall('.//MeshHeading/DescriptorName')]
                mesh_qualifier = [mesh_heading.text for mesh_heading in article.findall('.//MeshHeading/QualifierName')]
                
                data.append({
                    'pmid': pub_id,
                    'year': year,
                    'title': article_title,
                    'journal': journal_title,
                    'abstract': abstract,
                    'mesh_descriptor': mesh_descriptor,
                    'mesh_qualifier': mesh_qualifier
                })

# Convert data to a pandas DataFrame
df = pd.DataFrame(data)